In [5]:
# import dependencies
import csv
from numpy import array, asarray, zeros
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, SpatialDropout1D, LSTM
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
import pandas as pd
import re
from sklearn.model_selection import train_test_split


In [ ]:
# read the given imdb dataset.
# Download the imdb data set from
# https://data.world/popculture/imdb-5000-movie-dataset
# or try to search over kaggle

import pandas as pd
file_encoding = 'cp1252'   
pData=pd.read_csv('imdb_master.csv',encoding=file_encoding)

In [ ]:
pData.head()

In [ ]:
X=pData[['review','label']]

In [ ]:
X.label.unique()

## Process the data 

In [ ]:
X=X[X.label !='unsup']

In [ ]:
X.shape

In [ ]:
X.head()

In [3]:
# # Data Cleaning
X['review'] = X['review'].apply(lambda x: x.lower())
X['review'] = X['review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
X.head()

In [1]:
# maximum number of words to be selected from the complete corpous.
maxFeatures = 20000
tokenizer = Tokenizer(num_words=maxFeatures, split=' ')
tokenizer.fit_on_texts(X['review'].values)

# loading
import pickle

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer1 = pickle.load(handle)


# encodeDocuments = tokenizer1.texts_to_sequences(X['review'].values)


Using TensorFlow backend.


In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(encodeDocuments[30])

In [ ]:
print(tokenizer.document_count)    


In [ ]:
max_length = 200
paddedDocuments = pad_sequences(encodeDocuments, maxlen=max_length, padding='post')

In [ ]:
paddedDocuments[30]

In [ ]:
# split data
labels = []
for i in X['label']:
    if i == 'pos':
        labels.append(1)
    else:
        labels.append(0)

labels = array(labels)

In [ ]:
print(len(labels))


labels[25000:40000]


In [ ]:
import os
import numpy as np
# Globe word embedding
#download from 
# https://nlp.stanford.edu/projects/glove/

emb_dict = {}
glove = open(os.path.join("",'glove.6B.200d.txt'),encoding='utf-8')
for line in glove:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    emb_dict[word] = vector
glove.close()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(paddedDocuments,labels, test_size = 0.30, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
# create coefficient matrix for training data
trainingToEmbeddings = zeros((maxFeatures, 200))
for word, i in tokenizer.word_index.items():
    if i < 20000:
        gloveVector = emb_dict.get(word)
        if gloveVector is not None:
            trainingToEmbeddings[i] = gloveVector

In [ ]:
trainingToEmbeddings[10]
print(len(trainingToEmbeddings))

In [ ]:
model = Sequential()
model.add(Embedding(maxFeatures, 200, weights=[trainingToEmbeddings], input_length=max_length, trainable=False))
model.add(LSTM(200)) 
# model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs=5, batch_size=batch_size, verbose=0)

#### The output of the Embedding layer is a 2D vector with one embedding for each word in the input sequence of words (input document).

####  If you wish to connect a Dense layer directly to an Embedding layer, you must first flatten the 2D output matrix to a 1D vector using the Flatten layer.

In [2]:
from keras.models import load_model

model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print('Accuracy: %f' % (accuracy * 100))

In [3]:
new=[['i hate this particular movie becauase of its poor direction and bad location , even the casting of the actors is not proper'],
     ['movie was fucking awesome i love every single second'],
     ['this movie is really awesome'],
     ['i really liked the movie and had fun'],
    ["A powerful, funny, bitter and hopeful drama about love and friendship."],
    ["I think this film has enough going for it to deserve a recommendation, even as it inevitably fails to recapture the full measure of energy and wonder of the original. "]]

In [7]:
f=[]
max_length = 200
for i in new:
    tokenizer1.fit_on_texts(i)
    enc = tokenizer1.texts_to_sequences(i)
    enc=pad_sequences(enc, maxlen=max_length, padding='post')
    print(enc)
    
    f.append(enc)
    


[[   9  731   10  834   17    4   29  325  463    2   82 1450   54    1
  1034    4    1  150    6   21 2177    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [ ]:
f

In [ ]:
X_train[10]

In [14]:
m=model.predict(f[5])

In [15]:
m

array([[ 0.6582396]], dtype=float32)

In [ ]:
sing=X_train[:10]

In [ ]:
for i in sing:
    p=model.predict(np.array([i]))
    print(p)

In [ ]:
n=np.array(X_train[10])

In [ ]:
n.shape

In [ ]:
X_train.shape

###  Lets try out the saved model for prediction 

In [ ]:
from keras.models import load_model
model1 = load_model('my_model.h5')

In [ ]:
p=model1.predict(f[4])

In [ ]:
p

In [ ]:
p1=model1.predict(f[5])

In [ ]:
p1